This script aims to calculate the sea-air methane flux from in-situ data.

Air-sea CH4 exchange in Port Foster, was calculated across the sampled stations according to the following equation:

$F\ =\ k\ (C_W\ -{\ C}_A)\ \ \ \ (Eq.1)$

where F is the atmosphere-ocean CH4 flux (in µmol m-2 d-1), k is the gas transfer velocity (in cm h-1), Cw is the CH4 concentration in water and Ca is the equilibrium concentration of CH4 in the layer of air above the water. In equation 1, k  was estimated according to (Wanninkhof, 2014):

$k\ =\ a\ {<U10>}^2\left(\frac{{Sc}_{CH4}}{660}\right)^{-0.5}\ \ \ \ (Eq.\ 2)$

where a is a cost function used for gas transfer coefficient optimization (0.251), U10 is in situ the average winds (AEMET, 2021), averaged for 31 days (15 before and 15 after the sample was taken) and corrected for 10m height (m s-1) (Wanninkhof, 2014) and ScCH4 is the Schmidt number adjusted for temperature. The atmospheric equilibrium solubility of CH4 was derived from the following equation (Weiss, 1970; Wiesenburg and Guinasso, 1979):

$\ln{C_a}=\ln{f_G}+A_1+A_2\left(\frac{100}{T}\right)+A_3\ln{\left(\frac{T}{100}\right)}+A_4\left(\frac{T}{100}\right)+S‰[B1+B2{\left(\frac{T}{100}\right)}+B3{\left(\frac{T}{100}\right)}^2]\ \ \ \ (Eq.3)$

where fG is the molecular fraction in dry air for methane (1.89×10-6 as of January, 2021), T is the temperature in Kelvin and S is the salinity in ppt. Ai and Bi are the constants for calculation of solubilities in nmol L-1: A1 = -415.2807; A2 = 596.8104; A3 = 379.2599; A4 = -62.0757; B1 = -0.059160; B2 = 0.032174; B3 = -0.0048198 (Wiesenburg and Guinasso, 1979). The fourth temperature dependant parameter $A4{\left(\frac{T}{100}\right)}$ especially accounts for the vapor pressure of water. Additionally, methane saturation ratios were calculated as CW/CA × 100.

In [2]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

def calculate_average_wind_speed(file_path):
    # Load the JSON data from the specified file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Extract the wind speeds and timestamps
    wind_speeds = []
    timestamps = set()  # To track unique timestamps
    
    for i, entry in enumerate(data):
        if i % 3 != 0:  # Skip the first dictionary of each 10-minute interval
            timestamp = entry['fhora']
            if timestamp not in timestamps:
                wind_speed = entry['vel']
                if wind_speed != 'NaN':  # Ensure the wind speed is a valid number
                    wind_speeds.append(float(wind_speed))
                timestamps.add(timestamp)
    
    # Calculate the average wind speed
    if wind_speeds:
        average_wind_speed = sum(wind_speeds) / len(wind_speeds)
    else:
        average_wind_speed = 0

    return average_wind_speed

def function_x(bulk_c, temperature_C, salinity, wind_speed, stations, anem_height):

    '''
    Gas transfer velocity calculation (k) function

    Parameters:

    bulk_c: concentration in nmol/L of methane in water
    temperature_c: water temperature in ºC
    salinity: water salinity in PSU
    anem_height: the height of the meteo station at which the wind veolicty is recorded
    wind_speed: wind speed data recorded at the anemometer
    stations: number of the station

    Output:
    CH4 sea-water flux in umol/(m^2*d)
    '''
    
    # following the Equation 2: k = a * U10^2 * (ScCH4/Sc_ref)^b

    a = 0.251  # cm/h, from Wanninkhof, 2014.
    h = anem_height # effective height of the anemometer above the mean sea level
    Vh = wind_speed
    U10 = Vh/((h/10)**0.13)  
    b = -0.5
    Sc_ref = 660  # Reference Schmidt number

    # CH4 Schmidt number calculation
    # Constants for CH4 obtained from Wanninkhof, 2014 (Table 1 - Seawater)

    A = 2102.2
    B = (-131.54)
    C = 4.4931
    D = (-0.08676)
    E = 0.00070663
    t = temperature_C

    Sc_CH4 = A + B*t + C*t**2 + D*t**3 + E*t**4
    #print("Sc_CH4_calc =", round(Sc_CH4_calc,4))

    # Calculate the gas transfer velocity for CH4 (k)
    k = a * (U10 ** 2) * ((Sc_CH4 / Sc_ref) ** b)  # cm/h

    #print("k =", round(k,4))

    #MOIST AIR, constants obtained from Wiesenburg & Guinasso, 1979 (Table VI - nmol/L)
    A1 = -415.2807
    A2 = 596.8104
    A3 = 379.2599
    A4 = -62.0757
    B1 = -0.059160
    B2 = 0.032174
    B3 = -0.0048198
    S = salinity
    T = 273.15+(temperature_C)
    fg = 1.89*10**(-6) #2021 methane fraction in atmosphere

    # Calculate each term of the equation
    term1 = A1
    term2 = A2 * (100 / T)
    term3 = A3 * np.log(T / 100)
    term4 = A4 * (T / 100)
    term5 = S * (B1 + B2 * (T / 100) + B3 * (T / 100)**2)

    # Calculate ln(beta) which is the sum of all terms (Eq. 3)
    K0 = np.exp(np.log(fg) + term1 + term2 + term3 + term4 + term5)
    #print('K0 = ', K0)

    # Calculate the methane flux F in mol/(m^2 * s)
    Cw_nM = bulk_c
    F = k * (Cw_nM - K0)  
    flux = F * 0.24 # umol/(m^2 * d)

    #print("flux =", flux)

    return (stations,flux)

def process_lists_from_csv(csv_file_path, average_speed):
    # Load data from the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Extract columns into lists
    bulk_c_list = df['nM'].tolist()
    temperature_C_list = df['Temp'].tolist()
    salinity_list = df['Sal'].tolist()
    stations = df['station'].tolist()
    
     # Iterate over the elements of the lists simultaneously
    fluxes = []
    stations_f = []
    for a, b, c, e in zip(bulk_c_list, temperature_C_list, salinity_list, stations):
        # Call function_x with the corresponding elements and the single average_speed
        fluxes.append(round(function_x(a, b, c, average_speed, e, 12)[1], 2))
        stations_f.append(function_x(a, b, c, average_speed, e, 12)[0])

    return fluxes, stations_f

# Path to wind data file
wind_speed_file = './CH4_data/CH4_WIND_DATA.json'
average_speed = calculate_average_wind_speed(wind_speed_file)
print('Average wind speed: {}'.format(average_speed))

# Path to CH4 data file
ch4_data_file = './CH4_data/CH4_data.csv'
ch4_fluxes = process_lists_from_csv(ch4_data_file, average_speed)
for x in range(0,15):
    print('{} = {} umol/(m^2*d)'.format(ch4_fluxes[1][x],ch4_fluxes[0][x]))

# Create a DataFrame from the fluxes and stations
df = pd.DataFrame({
    'Station': ch4_fluxes[1],
    'Flux (umol/m^2*d)': ch4_fluxes[0]
})

# Save the DataFrame to a CSV file
output_file_path = './CH4_data/CH4_fluxes.csv'
df.to_csv(output_file_path, index=False)




Average wind speed: 7.22887570773793
St. 2-1 = 7.78 umol/(m^2*d)
St. 2-2 = 9.98 umol/(m^2*d)
St. 2-3 = 10.38 umol/(m^2*d)
St. 2-4 = 9.45 umol/(m^2*d)
St. 2-5 = 7.2 umol/(m^2*d)
St. 2-6 = 6.32 umol/(m^2*d)
St. 2-7 = 7.72 umol/(m^2*d)
St. 2-8 = 10.93 umol/(m^2*d)
St. 2-9 = 10.86 umol/(m^2*d)
St. 2-10 = 7.13 umol/(m^2*d)
St. 2-11 = 6.12 umol/(m^2*d)
St. 2-12 = 22.14 umol/(m^2*d)
St. 2-13 = 17.7 umol/(m^2*d)
St. 2-14 = 11.72 umol/(m^2*d)
St. 2-15 = 8.85 umol/(m^2*d)


PermissionError: [Errno 13] Permission denied: './CH4_data/CH4_fluxes.csv'